## Import Modul pendukung

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import xgboost as xgb
import os

## Import Dataset untuk Sample Machine Learningnya

In [9]:
df = pd.read_csv('weather_classification_data.csv')
df

## Inisialisasi untuk data yang berbentuk text / kata
Mengubah Table yang berisi text / kata agar nantinya dapat di baca oleh sistem. mengubah pada semua table yang ada text / kata nya, ke bentuk numerik (0,1,2-seterusnya)

In [ ]:
encoder = LabelEncoder()

### Melakukan Transformasi untuk bagian Feature Data 

In [ ]:
df['Cloud Cover']   = encoder.fit_transform(df['Cloud Cover'])
df['Location']      = encoder.fit_transform(df['Location'])
df['Season']        = encoder.fit_transform(df['Season'])

### Melakukan Transformasi untuk bagian Target Data

In [ ]:
y_target = encoder.fit_transform(df["Weather Type"])

### Melakukan pemisahan dataset dari Target Datanya

In [ ]:
dataset = df[[i for i in df.columns[:-1]]]
dataset

### Melakukan pembagian dataset
Pembagian untuk **Training Data** dan **Test Data**

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(dataset, 
                                                    y_target,
                                                    test_size=0.2,
                                                    random_state=50)

## Menentukan Parameter untuk membuat Model ML

- **objective: 'multi:softmax'**
  Untuk menentukan jenis klasifikasi. **'multi:softmax'** berarti model menggunakan softmax untuk klasifikasi multikelas.
- **num_class: len(df.columns[:-1])**
  Untuk menentukan jumlah kelas dalam masalah klasifikasi. **df.columns[:-1]** mengambil semua kolom kecuali kolom terakhir (biasanya label/target).
- **eta: 0.01**
  Untuk Learning rate, juga dikenal sebagai step size shrinkage. Nilai kecil (misalnya 0.01) membantu model belajar lebih lambat tetapi lebih stabil.
- **max_depth: 6**
  Untuk menentukan kedalaman maksimum pohon keputusan dalam model. Kedalaman yang lebih besar dapat meningkatkan kapasitas model tetapi juga meningkatkan risiko overfitting.

In [ ]:
parameters = {
    'objective':'multi:softmax',
    'num_class': len(df.columns[:-1]),
    'eta': 0.01,
    'max_depth': 6,
}

## Melakukan pembuatan Model Machine Learning dengan XGBClassifier

In [ ]:
model = xgb.XGBClassifier(
    parameters
)

In [ ]:
model.fit(x_train, y_train)
model.save_model("XGBClassifier.json")

In [ ]:
pred = model.predict(x_test)

# Hitung akurasi prediksi
accuracy = accuracy_score(y_test, pred)
print("Accuracy:", accuracy)

pred = encoder.inverse_transform(pred)
y_test = encoder.inverse_transform(y_test)

In [ ]:
database_salah = []
database_benar = []
db = []

for prediction, target in zip(pred, y_test):
    data = f'Prediction: {prediction}\tTarget: {target}'
    if prediction != target:
        db.append('Salah')
        database_salah.append(data)
    else:
        db.append('Benar')
        database_benar.append(data)

data = zip(
    np.array(pred),
    np.array(y_test),
    np.array(db)
)

dfPrediction = pd.DataFrame(data, columns=['Prediction', 'Target','Salah/Benar'])
print(f'Jumalah data Precision salah: {len(database_salah)}')
print(f'Jumalah data Precision benar: {len(database_benar)}')